# Request Units (RU) in Azure Cosmos DB
In this notebook, we'll learn about [Request Units (RU)](https://docs.microsoft.com/azure/cosmos-db/request-units), the unit of provisioned throughput in Azure Cosmos DB. 

Choosing the right amount of RU/s for your databases and containers is essential to optimizing cost and performance. We'll cover what are RU, how to provision RU/s for a database or container, and how to measure the RU cost of operations.


### What are Request Units (RU)?

Azure Cosmos DB works on a provisioned throughput model, where you set how much throughput you want for your databases or containers. Throughput is defined as Request Units/second, or RU/s. 

You can think of RU/s as the currency for throughput, or a rate-based currency. An RU is an abstract unit of compute that abstracts how much resources (CPU, IOPS, memory) are required for each operation. 

![RequestUnits](https://cosmosnotebooksdata.blob.core.windows.net/notebookdata/request-units.png)



### Understanding the benchmark for RU cost of operations

In Azure Cosmos DB, we've defined RU as follows:

##### 1 RU = cost to read a 1 KB document
##### 5 RU = cost to write a 1 KB document
##### Query = depends on number of documents returned and complexity of query

For example, let's say a container is provisioned at 400 RU/s, with 1 KB documents. Each second, Azure Cosmos DB can serve 400 reads / second, or 80 writes / second, or any combination of operations that consume up to 400 RU/s. 


### What happens when you exceed the provisioned RU/s?

Think of RU/s as a rate based currency. Each second, you have a certain RU/s to spend on operations. When you exceed the provisioned RU/s, the service will issue a 429 response along with a ```x-ms-retry-after-ms``` header to the client to indicate the request was throttled. By default, the Cosmos client SDKs handle these 429s and will retry up to 30 seconds or 9 times, whichever occurs first.

<img src="https://cosmosnotebooksdata.blob.core.windows.net/notebookdata/RequestUnitRateLimiting.PNG" alt="Explanation of rate limiting" width="50%"/>



### How to provision throughput (RU/s) on a database or container
Throughput can be provisioned at the database or container level. When it's provisioned at the database level, all containers in the database will share the throughput. When it's provisioned at the container level, each container gets a dedicated amount of throughput.



#### Create a new database with shared throughput

We'll create a new database with 400 RU/s, the minimum RU/s for a database with shared throughput in Azure Cosmos DB. Then we'll add four containers to it, which will all share the provisioned throughput. For every container added after the first four, there will be a minimum of 100 RU/s required per container.



In [20]:
from azure.cosmos.partition_key import PartitionKey

# Create a new database if it doesn't already exist
database_id = 'CosmosDBLevelRU'

database = cosmos_client.create_database_if_not_exists(id=database_id, offer_throughput=400)
print('Database with id \'{0}\' created'.format(database_id))

# Now we have a reference to the database we can use
database = cosmos_client.get_database_client(database_id)

# Delete existing containers
containers = database.list_containers()
for container in containers:
    database.delete_container(container['id'])    
    print('Existing container with id \'{0}\' dropped'.format(container['id']))

# Create new containers
for i in range(4):
    new_container_id = 'container_' + str(i)
    database.create_container_if_not_exists(id=new_container_id, partition_key=PartitionKey(path="/id"))
    print('Created container with id \'{0}\''.format(new_container_id))


Database with id 'CosmosDBLevelRU' created
Created container with id 'container_0'
Created container with id 'container_1'
Created container with id 'container_2'


In general, you should use shared database-level throughput when you don't have strict requirements around dedicated throughput for individual containers. For example, you may use shared database-level throughput for multi-tenant systems, which each tenant gets a container that shares the resources of the overall database, or for test, staging, or UAT envionrments that are accessed occasionally.


#### Create a new container with dedicated throughput

We'll create a new container with dedicated throughput of 1000 RU/s.

In [ ]:
from azure.cosmos.partition_key import PartitionKey

database_id = "CosmosDatabase"
container_id = "CosmosContainerDedicatedRU"
custom_offer_throughput = 1000

database = cosmos_client.create_database_if_not_exists(id=database_id)
print('Database created with id \'{0}\''.format(database_id))

# Now we have a reference to the database we can use
database = cosmos_client.get_database_client(database_id)

container = database.create_container_if_not_exists(id=container_id, partition_key=PartitionKey(path="/id"), offer_throughput=custom_offer_throughput)
print('Container with id \'{0}\' created with \'{1}\' RU/s'.format(container_id, custom_offer_throughput))

# Now we have a reference to the container we can use
container = database.get_container_client(container_id)

#### Verify the container was created in **Scale & Settings**.

![ScaleAndSettings](https://cosmosnotebooksdata.blob.core.windows.net/notebookdata/ScaleAndSettings.png)




### Scaling up the RU/s
You can change the RU/s on a database or container programatically to match your throughput requirements. For example, if you have a workload with high volume of traffic during business hours, but low traffic outside, you can scale up the RU/s during the day, and scale back down afterward. 


#### Get the current RU/s on a container

In [ ]:
# Get container object
database_id = "CosmosDatabase"
container_id = "CosmosContainerDedicatedRU"

container = cosmos_client.get_database_client(database_id).get_container_client(container_id)

# Get the current provisioned throughput
offer = container.read_offer()
provisioned_throughput = offer.offer_throughput

print('Container \'{0}\' has provisioned throughput: \'{1}\' RU/s'.format(container_id, provisioned_throughput))

#### Increase the RU/s on a container

In [ ]:
# Now, let's increase the RU on the container.
print('Now we programatically change the RU/s on the container...\n')
new_provisioned_throughput = min(provisioned_throughput + 100, 2000) # We'll increment in units of 100, up to 1000 RU/s. 
container.replace_throughput(new_provisioned_throughput)

print('Container \'{0}\' now has provisioned throughput: \'{1}\' RU/s'.format(container_id, container.read_offer().offer_throughput))

### Getting the RUs for any operation in Azure Cosmos DB

The Azure Cosmos DB provides the exact RU cost for any operation performed against the service. We can access this via the ```x-ms-request-charge``` header. For example, we'll write a new document to our container and measure the RU charge.

In [ ]:
### Write a document to the container
import uuid

task_document_body = {'id': str(uuid.uuid4()), 'task': 'Learn Azure Cosmos DB!', 'status': 'In-progress'}
task_document_response = container.create_item(body=task_document_body)

print('Created document with id \'{0}\''.format(task_document_response['id']))
print('Request charge: \'{0}\' RUs'.format(container.client_connection.last_response_headers['x-ms-request-charge']))

### Next steps
Now that you've learned about RU/s in Azure Cosmos DB, here are some more resources to check out:

[Provisioned Throughput](https://docs.microsoft.com/azure/cosmos-db/set-throughput)

[Estimating RU for a workload](https://docs.microsoft.com/azure/cosmos-db/estimate-ru-with-capacity-planner)

[Find the RU charge for operations](https://docs.microsoft.com/azure/cosmos-db/find-request-unit-charge)
